# Cleaning Redfin Data for Investigative Journalists

This notebook walks you through cleaning a Redfin 'Sold Homes' dataset using Python and pandas. The goal is to prepare the data for analysis related to housing trends and potential gentrification patterns.

## Step 1: Import libraries

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Step 2: Import csv file

In [ ]:
from google.colab import files
uploaded = files.upload()

### Step 3: Check the structure of the data
Let's explore the columns and data types to understand what we have.

In [ ]:

# Automatically read the uploaded CSV
filename = list(uploaded.keys())[0]  # Get the first uploaded filename
df = pd.read_csv(filename)

# Show structure
print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())
print("\nData Preview:")
df.head()


### Step 4: Rename columns for readability
We'll simplify long, tool-generated column names to more journalist-friendly ones.

In [ ]:
df.columns = [
    "URL", "Sold_Label", "Address", "Listing_Details", "Price",
    "Beds", "Baths", "SqFt"
]
df = df[[
    "URL", "Sold_Label", "Address", "Listing_Details", "Price",
    "Beds", "Baths", "SqFt"
]]
df.head()

### Step 5: Clean numeric fields (price, beds, baths, sqft)
We'll remove any symbols like `$` or `,` and convert to numeric values for analysis.

In [ ]:
df['Beds'] = df['Beds'].astype(str)
df['Baths'] = df['Baths'].astype(str)
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
df['Beds'] =  df['Beds'].str.extract(r'(\d+\.?\d*)').astype(float)
df['Baths'] = df['Baths'].str.extract(r'(\d+\.?\d*)').astype(float)
df['SqFt'] = df['SqFt'].replace({',': '', '‚Äî': None}, regex=True)
df['SqFt'] = pd.to_numeric(df['SqFt'], errors='coerce')

### Step 6: Extract sold date from the label
This gives us a usable `datetime` column for filtering or time-based analysis.

In [ ]:
df['Sold_Date'] = df['Sold_Label'].str.extract(r'SOLD(?: BY REDFIN)? ([A-Z]{3} \d{1,2}, \d{4})')
df['Sold_Date'] = pd.to_datetime(df['Sold_Date'], format='%b %d, %Y', errors='coerce')
df = df.drop_duplicates()

df.head()

### Step 7: Calculate price per square foot
This helps spot overvalued flips or gentrification signals.

In [ ]:
df['Price_per_SqFt'] = df['Price'] / df['SqFt']

### Step 8: Extract ZIP code from the address
This enables location-based filtering and grouping.

In [ ]:
df['ZIP'] = df['Address'].str.extract(r'(\d{5})')

### Final Cleaned Dataset Preview
You're now ready to begin your analysis.

In [ ]:
# Convert Price column to numeric
df['Price'] = pd.to_numeric(df['Price'].replace('[\$,]', '', regex=True), errors='coerce')

# Remove sold_label column
df = df.drop('Sold_Label', axis=1)

In [ ]:
df.head()

# Additional Analysis: Trends and Potential Flips

### Step 1: Choose a ZIP code to explore

In [ ]:
# Show ZIP codes available
df['ZIP'].value_counts().head(10)

In [ ]:
# Set your focus ZIP code
target_zip = '30349'  # Change this to any ZIP code in your dataset
focus_df = df[df['ZIP'] == target_zip]

### Step 2: Price Trends Over Time

In [ ]:
# Group by month to see trend
focus_df['Month'] = focus_df['Sold_Date'].dt.to_period('M')
monthly_prices = focus_df.groupby('Month')['Price'].mean()
monthly_prices.plot(title=f'Average Sale Price in {target_zip}', marker='o')
plt.ylabel('Price')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

### Step 3: Price Per Square Foot Distribution

In [ ]:
sns.histplot(focus_df['Price_per_SqFt'].dropna(), bins=20, kde=True)
plt.title(f'Price per Sq Ft in ZIP {target_zip}')
plt.xlabel('Price/SqFt')
plt.grid(True)
plt.show()

### Step 4: Identify Possible Flips
A flip might be a small house (under 1500 sqft) sold for a very high price per sqft.


In [ ]:
flips = focus_df[(focus_df['SqFt'] < 1500) & (focus_df['Price_per_SqFt'] > 175)]
flips[['Address', 'Price', 'SqFt', 'Price_per_SqFt', 'Sold_Date']]

## Step 5: Analyze trens by zipcode

In [ ]:
# Create a 'Year' column
df['Year'] = df['Sold_Date'].dt.year

# Group and calculate average price per year per ZIP
yearly_prices = df.groupby(['ZIP', 'Year'])['Price'].mean().reset_index()


In [ ]:
plt.figure(figsize=(12, 6))


for zip_code in yearly_prices['ZIP'].unique():
    zip_data = yearly_prices[yearly_prices['ZIP'] == zip_code]
    plt.plot(zip_data['Year'], zip_data['Price'], label=f'ZIP {zip_code}')

plt.title('Yearly Average Home Prices by ZIP Code')
plt.xlabel('Year')
plt.ylabel('Avg Price ($)')
plt.xticks(sorted(yearly_prices['Year'].unique()))  # Use clean integer years
plt.grid(True)
plt.tight_layout()
plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1), ncol=1)
plt.show()